# Object Detection Model Training & Evaluation

**Author:** Erick Lemmy dos Santos Oliveira

**GitHub:** [Slots Detection](https://github.com/eriklemy/slots_detection)

# Introduction

This project implements an object detection system to detect Slots for planting flowers. Despite the limited dataset size, various techniques where tested to create a detection model capable of performing effectively.

### Project Objectives
1. Train an object detection model with the data (Yolo or any other model).
2. Evaluate the model performance (select and justify evaluation metrics).
3. Discuss the model performance and errors.
4. Discuss the steps used to solve the problem.
5. Suggest improvements in data/model/etc.

### Dataset Characteristics
- Classes: 1 (Slots)
- Total images: 21
- split:
    - train: 16 images (75%)
    - val: 5 images    (25%)

### Technical Environment
- Hardware: Google Colab GPU (NVIDIA T4/V100)
- Framework: Ultralytics YOLO
- Acceleration: CUDA-enabled training

In [ ]:
%pip install ultralytics sahi albumentations

import ultralytics
ultralytics.checks()

Run the necessary imports bellow

In [ ]:
import os
import cv2
import glob
import torch

import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

from ultralytics import YOLO
import matplotlib.pyplot as plt
from collections import defaultdict


%matplotlib inline

**Verify NVIDIA GPU Availability**

Checking for the avaliability of the GPU. If shows its off, go to "Runtime" -> "Change Runtime type" in the top menu bar, and then selecting one of the GPU options in the Hardware accelerator section.

In [ ]:
!nvidia-smi

print(f"CUDA available: {torch.cuda.is_available()}")
device = 0 if torch.cuda.is_available() else 'cpu'

#1.&nbsp; Dataset Verification



In [ ]:
from google.colab import drive

!unzip "/content/drive/MyDrive/erick.zip" -d "/content/datasets"


### **Pre-Training Dataset Evaluation**  

Before training, it is essential to evaluate the dataset by verifying that both images and labels are correct. This process also helps in identifying a suitable model based on the dataset format, reducing the need for significant modifications if it already aligns with a specific model.  

### **Reasons:**  

1. **Prevent Training Failures**  
   - YOLO coordinates outside the [0,1] range can cause numerical errors (e.g., NaN loss).  
   - Incorrect folder structure can prevent data from loading properly.  

2. **Ensure Annotation Quality**  
   - Poor annotations can reduce mAP scores.  
   - Unlabeled objects introduce bias, leading to inaccurate learning.  

3. **YOLO Compatibility**  
   - Requires a specific directory format:  
     ```bash
     dataset/
     ├── train/images/  # .jpg, .png
     └── train/labels/  # .txt (one file per image)
     ```  
   - Class IDs must match the YAML configuration file.  

The provided dataset follows the standard structure, with separate training and validation sets, as well as distinct folders for images and labels. Since it includes a YAML file and annotations in `.txt` format structured as:  
   - (`class`, `center_x`, `center_y`, `height`, `width`)  
   - Values normalized between **0 and 1**  

This suggests that the dataset was created using an auxiliary labeling tool, such as [Roboflow](https://roboflow.com), and later converted to YOLO format. By visualizing an image alongside its corresponding annotations, the format was confirmed.**negrito**

In [ ]:
def plot_label_check(img, title):
    sample_img = cv2.imread(img_path)
    annotated_img = sample_img.copy()

    h, w = sample_img.shape[:2]
    label_path = img_path.replace("images", "labels").replace(".jpg", ".txt")
    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.split())

            x_center, y_center = int(x_center * w), int(y_center * h)
            width, height = int(width * w), int(height * h)

            x1, y1 = x_center - width // 2, y_center - height // 2
            x2, y2 = x1 + width, y1 + height

            cv2.rectangle(annotated_img, (x1, y1), (x2, y2), (255, 0, 0), 2)

    fig, ax = plt.subplots(1, 2, figsize=(20, 7))

    ax[0].imshow(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    ax[1].imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
    ax[1].set_title("Annotated Image")
    ax[1].axis("off")

    plt.tight_layout()
    plt.suptitle(f"{title}: {os.path.basename(img_path)}")
    plt.show()

img_path = "datasets/erick/val/images/7126_18_7281_71-6_2ND.jpg"
plot_label_check(img_path, "Sample Image with Bounding Box")

#2.&nbsp;Image Resolution and Color Analysis

### **Visualizing Image Resolutions**  

Analyzing the distribution of image resolutions in the **training** and **validation** sets helps identify potential imbalances that could affect model training and generalization.  

### **Methodology:**  
1. **Extract Resolutions:** Collect width and height from all images.  
2. **Count Occurrences:** Determine the number of unique resolutions and the number of images per resolution.  

### **Observations:**  

1. **Training Set Imbalance:**  
   - 80% of the training images are concentrated in just **two resolutions** (`4000x2252` and `4032x3024`).  
   - Resolutions like `1600x900` and `2000x1126` contain only **one image each**, which could lead to *overfitting* for these specific formats.  

2. **Limited Validation Coverage:**  
   - The validation set covers only **50%** of the resolutions found in the training set.  
   - The resolution `4032x3024` has only **one image** in validation, making evaluation unreliable for this format.  

### **Implications:**  
- **Resolution Bias:** The model may perform poorly on underrepresented resolutions (e.g., `1600x900`).  
- **Preprocessing Needs:** Different resolutions require strategies like *resizing* or *padding* for consistency.  
- **Generalization Issues:** The validation set does not sufficiently cover the training resolutions, increasing performance drops in real-world deployment.

In [ ]:
def get_resolution_and_color_means(image_dir):
    resolutions = []
    color_means = []

    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        img = cv2.imread(img_path)

        if img is not None:
            # Get resolution
            h, w = img.shape[:2]
            resolutions.append((w, h))

            # Calculate color means (BGR channels)
            color_means.append(img.mean(axis=(0, 1)))
    return resolutions, color_means

train_image_dir = "datasets/erick/train/images"
train_label_dir = "datasets/erick/train/labels"

val_image_dir = "datasets/erick/val/images"
val_label_dir = "datasets/erick/val/images"

resolutions_train, color_means_train = get_resolution_and_color_means(train_image_dir)
resolutions_val, color_means_val = get_resolution_and_color_means(val_image_dir)

print(f"========= Resolutions =========")
print(f"Number of unique resolutions (train): {len(set(resolutions_train))}")
print(f"Images per resolution (train): {dict((res, resolutions_train.count(res)) for res in set(resolutions_train))}")
print(f"Number of unique resolutions (val): {len(set(resolutions_val))}")
print(f"Images per resolution (val): {dict((res, resolutions_val.count(res)) for res in set(resolutions_val))}")

### Analyze Color Channels

Comparing the statistical distributions (mean and standard deviation) of the color channel intensities (BGR) between the training and validation sets to ensure consistency and identifying potential biases that may affect the model.

### **Methodology:**  
1. **Calculation of Statistics:**  
   - Mean and standard deviation of the average intensities for each channel (Blue, Green, Red) across all images.  
2. **Visualization:**  
   - Histograms to compare intensity distributions.  
   - Boxplots to analyze variation and outliers.  

### **Observations:**
The original dataset does not exhibit many outliers and appears consistent with the expected scenario. The high Green and Red values are due to the presence of plants and ground textures, while Blue is typically less prominent in natural environments. As a result, there is no significant indication of bias. However, applying a Hue augmentation could help simulate greater color variability and further enhance the model's robustness.


In [ ]:
color_means_train = np.array(color_means_train)
color_means_val = np.array(color_means_val)

print(f"========= Color Channel Statistics =========")
print(f"Color means -> B: {color_means_train[:, 0].mean()}, G: {color_means_train[:, 1].mean()}, R: {color_means_train[:, 2].mean()}")
print(f"Color std -> B: {color_means_train[:, 0].std()}, G: {color_means_train[:, 1].std()}, R: {color_means_train[:, 2].std()}")

In [ ]:
def plot_color_analysis(color_means, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

    # Plot histogram
    ax1.hist(color_means[:, 0], bins=50, alpha=0.5, color='b', label='Blue')
    ax1.hist(color_means[:, 1], bins=50, alpha=0.5, color='g', label='Green')
    ax1.hist(color_means[:, 2], bins=50, alpha=0.5, color='r', label='Red')
    ax1.set_title(f'Color Histogram - {title}')
    ax1.set_xlabel('Pixel Intensity Mean')
    ax1.set_ylabel('Frequency')
    ax1.legend()

    # Plot boxplot
    ax2.boxplot([color_means[:, 0], color_means[:, 1], color_means[:, 2]])
    ax2.set_title(f'Color Channel Distributions - {title}')
    ax2.set_xticks([1, 2, 3])
    ax2.set_xticklabels(['Blue', 'Green', 'Red'])
    ax2.set_ylabel('Pixel Intensity Mean')

    plt.tight_layout()
    plt.show()

plot_color_analysis(color_means_train, 'Train Dataset')
# plot_color_analysis(color_means_val, 'Validation Dataset')

In [ ]:
def analyze_dataset_statistics(image_dir, label_dir):
    """Analyze dataset statistics including bounding box metrics and class distribution.

    Returns:
        dict: Dictionary containing computed statistics
    """
    stats = {
        'bbox_areas': [],
        'bbox_aspect_ratios': [],
        'class_counts': defaultdict(int),
        'num_boxes_per_image': [],
        'missing_images': 0,
        'corrupted_files': 0
    }

    label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

    print(f"Analyzing {len(label_files)} label files...")

    for label_file in tqdm(label_files, desc="Processing images"):
        label_path = os.path.join(label_dir, label_file)
        image_file = os.path.splitext(label_file)[0] + ".jpg"
        image_path = os.path.join(image_dir, image_file)

        # Validate image existence
        if not os.path.exists(image_path):
            stats['missing_images'] += 1
            continue

        img = cv2.imread(image_path)
        if img is None:
            stats['corrupted_files'] += 1
            continue

        h, w = img.shape[:2]
        num_boxes = 0

        # Process labels
        with open(label_path, 'r') as f:
            for line in f:
                class_id, x_center, y_center, width, height = map(float, line.split())
                stats['class_counts'][int(class_id)] += 1

                # Convert to absolute coordinates
                width_abs = width * h
                height_abs = height * w

                # Calculate metrics
                area = width_abs * height_abs
                aspect_ratio = width_abs / height_abs if height_abs > 0 else 0

                stats['bbox_areas'].append(area)
                stats['bbox_aspect_ratios'].append(aspect_ratio)
                num_boxes += 1

        stats['num_boxes_per_image'].append(num_boxes)

    return stats

def print_statistics(stats):
    """Print formatted statistics"""
    total_images = len(stats['num_boxes_per_image'])
    total_boxes = sum(stats['class_counts'].values())

    print("\nDataset Statistics Report")
    print("============================")
    print(f"Total images analyzed: {total_images:,}")
    print(f"Total bounding boxes: {total_boxes:,}")
    print(f"Missing images: {stats['missing_images']}")
    print(f"files: {stats['corrupted_files']}\n")

    print("Bounding Box Metrics")
    print(f"- Average boxes per image: {np.mean(stats['num_boxes_per_image']):.2f}")
    print(f"- Area (pixels²) - Mean: {np.mean(stats['bbox_areas']):,.2f}, Median: {np.median(stats['bbox_areas']):,.2f}")
    print(f"- Aspect Ratio - Mean: {np.mean(stats['bbox_aspect_ratios']):.2f}, Median: {np.median(stats['bbox_aspect_ratios']):.2f}\n")

    print("Class Distribution")
    for class_id, count in sorted(stats['class_counts'].items()):
        print(f"  Class {class_id}: {count:,} boxes ({count/total_boxes:.1%})")

def plot_distributions(stats):
    """Visualize dataset distributions"""
    plt.figure(figsize=(12, 5))

    # Bbox Area
    plt.subplot(1, 2, 1)
    plt.hist(stats['bbox_areas'], bins=30, color='skyblue', edgecolor='black')
    plt.title('Bounding Box Area Distribution')
    plt.xlabel('Pixels²')
    plt.ylabel('Count')
    plt.yscale('log')

    # Aspect Ratio
    plt.subplot(1, 2, 2)
    plt.hist(stats['bbox_aspect_ratios'], bins=30, color='salmon', edgecolor='black')
    plt.title('Aspect Ratio Distribution')
    plt.xlabel('Width/Height Ratio')
    plt.ylabel('Count')


    plt.tight_layout()
    plt.show()

train_stats = analyze_dataset_statistics(train_image_dir, train_label_dir)
print_statistics(train_stats)

#3.&nbsp;Train Model

### Why YOLO?

The YOLO architecture was selected for its:

- Real-time inference capabilities
- Balance between accuracy and speed
- Strong performance on small datasets
- Extensive pre-trained weights availability
- Dataset Compatibility
- Agricultural Applications [[1](https://www.sciencedirect.com/science/article/pii/S2772375524002533?via%3Dihub), [2](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5090169), [3](https://onlinelibrary.wiley.com/doi/10.1155/2023/4039179), [4](https://www.mdpi.com/2073-4395/14/10/2194), [5](https://www.mdpi.com/2073-4395/14/8/1628)]
- SAHI compatibility        [[1](https://ieeexplore.ieee.org/document/9897990), [2](https://github.com/obss/sahi?tab=readme-ov-file)]
- Augmentation capabilities




## Training Parameters
With the model defined, the training starts! First, there are a few important parameters to decide on.

**Model architecture & size (`model`):**

Several YOLO versions exists, like yolov5, yolov8 and yolov11 with different models sizes (`yolov8n.pt`, `yolo8m.pt`, `yolo11n.pt`, `yolo11m.pt`, `yolo11x.pt`). Larger models run slower but have higher accuracy, while smaller models run faster but have lower accuracy. For this project the `yolov8n.pt` was chosen since it has less params than older models and provides balanced trade-off between speed and accuracy compared to others models like v9 (more computational heavy) and YOLO11 with is optimized for speed and efficiency, is not the best choice for this task since its focus is on detection smaller objects in higher resolution imagens. The decision to use the `n` variant was made to minimize/prevent overfitting during the transfer learning while ensuring that the model remains sufficiently capable of handling the complexity of small object detection.

**Model architecture & size (`model`):**  

Several YOLO versions exist, such as YOLOv5, YOLOv8, and YOLOv11, each offering models of varying sizes (`yolov8n.pt`, `yolov8m.pt`, `yolo11n.pt`, `yolo11m.pt`, `yolo11x.pt`). Larger models tend to run slower but offer higher accuracy, while smaller models run faster but sacrifice some accuracy. For this project, the `yolov8n.pt` model was selected. Although it has fewer parameters than the newer models (except YOLOv11), YOLOv8 provides a balanced trade-off between speed and accuracy. YOLOv11, while optimized for speed and efficiency, is not the best choice for this task since its focus is on maximizing performance in real-time scenarios, which is not the primary requirement here. YOLOv8, on the other hand, is well-suited for achieving good detection accuracy with reasonable processing time, making it ideal for the dataset and the computational constraints of this project.


**Number of epochs (`epochs`) and (`batchs`)**

With a limited dataset, an initial setting of 50 epochs is used to provide sufficient training while mitigating the risk of overfitting. YOLO automatically adjusts the batch size based on GPU memory, typically utilizing around 60% of the GPU’s RAM. For a resolution of 640x640, this usually results in a batch size between 8 and 16. This configuration strikes a balance between efficient training and resource utilization.


**Resolution (`imgsz`)**

Image Resolution has a large impact on the speed and accuracy of the mode. Lower resolution model will have higher speed but less accuracy. YOLO models are typically trained and inferenced at a 640x640 resolution.


**Data Augmentation**

Given the limited size of our dataset, we apply data augmentation to improve the model's generalization and robustness. YOLO provides built-in augmentation tools that eliminate the need to create entirely new datasets.
The following augmentation techniques are in use:
- Mosaic (1.0) – Set to 1.0 to ensure that all images will have that augmentation. This augmentation combines multiple images into one, helping the model learn object variations and different backgrounds.
- MixUp (0.5) – Set to 0.5 to ensure that this augmentation only happens 50% of the time. This augmentation blends two images and their labels, improving the model's ability to handle occlusions and uncertainties reducing overfitting.
- Scale (0.5) – Randomly resizes objects to enhance scale invariance. (50% probability to scale)
- Flip (fliplr: 0.5, flipud: 0.2) – Applies horizontal and vertical flips to introduce viewpoint variations, horizontal is particularly effective for overhead/aerial images like those in our datase. (Aplied to 50% and 20% of images)
- HSV Augmentation (hsv_h: 0.015, hsv_s: 0.9, hsv_v: 0.9) – Slightly hye Adjust to prevent extreme color shift, high saturation, and brightness to improve color invariance.


**Transfer Learning**

Using a pre-trained model leverages knowledge acquired from large datasets, facilitating the extraction of relevant features even if the target class isn't present in the original model. Freezing some of the initial layers preserves fundamental features, while the upper layers are fine-tuned to learn the new classes and adapt to our specific domain.

### Training
Run the following code block to begin training.

In [ ]:
model = YOLO("yolov8n.pt")

data_yaml_path = "/content/datasets/erick/data.yaml"

config = {
    "data": data_yaml_path,
    "epochs": 50,
    "imgsz": 640,
    "batch": 16,
    "workers": 8,      # Number of CPU workers (default: 8)
    "device": device,
    "freeze": list(range(11)), # 0 ... 10
    "mosaic": 1.0,
    "mixup": 0.5,
    "scale": 0.5,
    "fliplr": 0.5,
    "flipud": 0.2,
    "hsv_h": 0.015,
    "hsv_s": 0.9,
    "hsv_v": 0.9,
    "patience": 30,     # for early stop if theres no improvement
    "seed": 42,         # for reproducibility
    "single_cls": True, # for single class detection
    "project": "slots",
    "name": "standard",
}

results = model.train(**config)

## Model Evaluation

To evaluate the model's performance, the metrics **Precision**, **Recall**, **mAP** (Mean Average Precision), and **mAP@50-95** were selected. These metrics are widely used in object detection evaluations and are standard in frameworks like YOLO, as well as in renowned benchmarks such as COCO.

**Precision** and **Recall** provide essential insights into the quality of the detections by indicating the proportion of true positives relative to predictions and the model's ability to correctly identify objects. In contrast, **mAP@50-95** assesses the average precision of the model across different IoU (Intersection over Union) thresholds, offering a comprehensive view of detection accuracy at varying levels of overlap between predictions and ground truth objects.

In [ ]:
best_model = YOLO(f"/content/{config['project']}/{config['name']}/weights/best.pt")

metrics = best_model.val()
print("======== Metrics ========")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"Results: {metrics.box.p.mean():.4f}")
print(f"Recall: {metrics.box.r.mean():.4f}")

## Visualize Results

In [ ]:
results_path = Path(f"{config['project']}/{config['name']}/")
results = cv2.imread(str(results_path / "results.png"))
plt.figure(figsize=(14, 10))
plt.imshow(cv2.cvtColor(results, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

### Test Image

In [ ]:
sample_img = "/content/datasets/erick/val/images/7039_14_151-6_2nd.jpg"
results = best_model(sample_img)

res_plotted = results[0].plot(line_width=3)
plt.figure(figsize=(12, 5))
plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Detected Objects")
plt.show()

# 4.&nbsp;Improvement Strategy

## Slicing Strategy for Improving Small Object Detection

The original images had high resolutions (e.g., 4032x3024, 4000x2252), and the objects to be detected were very small relative to the overall image size:  
- **Average BBox area:** 18,936.92 pixels²  
- **Median BBox area:** 20,804.56 pixels²  

For reference, in a typical 4000x2252 image (~9MP):  
- The object occupied only **~0.2% of the total image area** (18,936 / 9,000,000 ≈ 0.0021).

### Identified Problem  
At such high resolutions, the object becomes **nearly imperceptible** to conventional detection models, even when using absolute bounding boxes. This results in:  
1. Loss of detail during downsampling (resizing to standard resolutions such as 640x640).  
2. Learning difficulties due to the low pixel density of the object.

### Implemented Solution  
The images were divided into **640x640 pixel patches** using a slicing technique, resulting in:  
- **Expansion of the training dataset:** from 16 to 581 images.  
- **Benefits:**  
  - **Artificial increase in the object's relative resolution:**  
    In a 640x640 patch (409,600 pixels²), the object's area now represents **~4.6%** (18,936 / 409,600) — making it 22 times more prominent.  
  - Reduced computational complexity, as there are fewer pixels to process per iteration.  
  - Generation of pseudo-augmented data that provides contextual variations.

### Using SAHI (Slicing Aided Hyper Inference) for Small Objects

#### How SAHI Complements YOLO
| **Approach**         | **SAHI Advantage**                   | **Limitation of Pure YOLO**      |
|----------------------|--------------------------------------|----------------------------------|
| **Inference**        | Processes the image in slices        | Processes the entire image       |
| **Object Size**      | Detects objects smaller than 50px    | Often misses smaller objects |
| **Computational Cost** | Increases processing time by ~20%, but improves mAP@50-95 by ~30% | Faster inference, but with lower precision |

This slicing strategy, especially when paired with SAHI, significantly enhances the detection of small objects by increasing their relative size and preserving more details during inference.

## Data Augmentation with Albumentations

The use of the Albumentations library, in conjunction with YOLO augmentations, allows for the generation of a larger and more robust dataset by providing techniques not present in YOLO or with limited configuration options. In this sense, the following augmentations were applied to the pipeline:


  ```python
    # Augmentation pipeline
    aug_transforms = A.Compose([
        A.RandomRotate90(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.HueSaturationValue(p=0.2),
        A.RandomGamma(gamma_limit=(60, 115), p=0.3),
        A.GaussNoise(p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
  ```

The levels of brightness, saturation, hue and GaussNoise were kept at their default values and adjusting just the probability to happen to , as they are already tuned for use with YOLO, while the RandomGamma required fine-tuning to better distribute the irregular exposure generated (dark/light). The use of Albumentations allowed for a 3x increase in the dataset size, totaling 1,940 images, with 1,743 for training and 197 for validation.



## Utils File
Run the block bellow to create the utils file

In [ ]:
%%writefile utils.py
import os
import cv2

import yaml
import shutil
import numpy as np
from tqdm import tqdm
from pathlib import Path

import albumentations as A
from ultralytics import YOLO

from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.predict import get_prediction, get_sliced_prediction, predict

import torch

device = 0 if torch.cuda.is_available() else 'cpu'

def load_yaml(yaml_file):
    """Load YAML file and return contents."""
    with open(yaml_file, 'r') as f:
        return yaml.safe_load(f)

def plot_training_results(results_path):
    """
    Load and plot the training results from the specified CSV file.
    """
    results = pd.read_csv(results_path)
    print(results.head())
    plt.figure(figsize=(10, 5))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(results['train/box_loss'], label='train/box_loss')
    plt.plot(results['val/box_loss'], label='val/box_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Box Loss')
    plt.legend()

    # mAP plot
    plt.subplot(1, 2, 2)
    plt.plot(results['metrics/mAP50(B)'], label='metrics/mAP50(B)')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.title('mAP')
    plt.legend()

    plt.tight_layout()
    plt.show()

def plot_color_histogram(color_means, title):
    plt.figure(figsize=(10, 5))
    plt.hist(color_means[:, 0], bins=50, alpha=0.5, color='b', label='Blue')
    plt.hist(color_means[:, 1], bins=50, alpha=0.5, color='g', label='Green')
    plt.hist(color_means[:, 2], bins=50, alpha=0.5, color='r', label='Red')
    plt.title(title)
    plt.xlabel('Pixel Intensity Mean')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

def plot_color_channel_distribution(color_means, title):
    plt.figure(figsize=(10, 5))
    plt.boxplot([color_means[:, 0], color_means[:, 1], color_means[:, 2]])
    plt.title('Color Channel Distributions (BGR)')
    plt.xticks([1, 2, 3], ['Blue', 'Green', 'Red'])
    plt.ylabel('Pixel Intensity Mean')
    plt.show()

def create_sahi_dataset(data_yaml_path, output_dir, slice_size=640, overlap_ratio=0.2):
    """
    Create a SAHI dataset from the original dataset.

    Args:
        data_yaml_path: Path to the data.yaml file
        output_dir: Directory to save the augmented dataset
        slice_size: Size of the slices
        overlap_ratio: Overlap ratio between slices
    """
    data_config = load_yaml(data_yaml_path)

    # Create output directories
    sahi_train_dir = os.path.join(output_dir, "train")
    sahi_val_dir = os.path.join(output_dir, "val")
    sahi_train_images_dir = os.path.join(sahi_train_dir, "images")
    sahi_train_labels_dir = os.path.join(sahi_train_dir, "labels")
    sahi_val_images_dir = os.path.join(sahi_val_dir, "images")
    sahi_val_labels_dir = os.path.join(sahi_val_dir, "labels")

    os.makedirs(sahi_train_images_dir, exist_ok=True)
    os.makedirs(sahi_train_labels_dir, exist_ok=True)
    os.makedirs(sahi_val_images_dir, exist_ok=True)
    os.makedirs(sahi_val_labels_dir, exist_ok=True)

    # Determine dataset structure
    dataset_dir = os.path.dirname(data_yaml_path)
    train_dir = os.path.join(dataset_dir, "train")
    val_dir = os.path.join(dataset_dir, "val")

    # Process training and validation images
    process_dataset_split(train_dir, sahi_train_images_dir, sahi_train_labels_dir, slice_size, overlap_ratio)
    process_dataset_split(val_dir, sahi_val_images_dir, sahi_val_labels_dir, slice_size, overlap_ratio)

    # Create a new data.yaml
    new_data_yaml = os.path.join(output_dir, "data.yaml")
    data_config['path'] = output_dir
    data_config['train'] = os.path.join(output_dir, "train", "images")
    data_config['val'] = os.path.join(output_dir, "val", "images")

    with open(new_data_yaml, 'w') as f:
        yaml.dump(data_config, f)

    print(f"SAHI-augmented dataset created at {output_dir}")
    print(f"New data.yaml file: {new_data_yaml}")

    return new_data_yaml

def process_dataset_split(source_dir, output_images_dir, output_labels_dir, slice_size, overlap_ratio):
    """
    Process a dataset split (train or val) with SAHI slicing.

    Args:
        source_dir: Source directory containing images and labels
        output_images_dir: Output directory for images
        output_labels_dir: Output directory for labels
        slice_size: Size of the slices
        overlap_ratio: Overlap ratio between slices
    """
    source_images_dir = os.path.join(source_dir, "images")
    source_labels_dir = os.path.join(source_dir, "labels")

    # If directories don't exist, assume images and labels are in the same directory
    if not os.path.exists(source_images_dir):
        source_images_dir = source_dir
        source_labels_dir = source_dir.replace("images", "labels")

    for img_file in tqdm(os.listdir(source_images_dir), desc=f"Processing {os.path.basename(source_dir)}"):
        if not (img_file.endswith('.jpg') or img_file.endswith('.png') or img_file.endswith('.jpeg')):
            continue

        img_path = os.path.join(source_images_dir, img_file)
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"
        label_path = os.path.join(source_labels_dir, label_file)

        shutil.copy(img_path, os.path.join(output_images_dir, img_file))
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(output_labels_dir, label_file))

        # Perform SAHI slicing
        try:
            image = cv2.imread(img_path)
            h, w = image.shape[:2]

            if h > slice_size or w > slice_size:  # Only slice if image is larger than slice size
                slice_and_save(img_path, label_path, output_images_dir, output_labels_dir,
                              slice_size, overlap_ratio, base_name)
        except Exception as e:
            print(f"Error processing {img_file}: {e}")
            continue

def slice_and_save(img_path, label_path, output_images_dir, output_labels_dir, slice_size, overlap_ratio, base_name):
    """
    Slice an image and its annotations using SAHI-style slicing and save to output directories.

    Args:
        img_path: Path to the input image
        label_path: Path to the input label (YOLO format)
        output_images_dir: Output directory for sliced images
        output_labels_dir: Output directory for sliced labels
        slice_size: Size of the slices
        overlap_ratio: Overlap ratio between slices
        base_name: Base name for the output files
    """
    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    # Get YOLO labels -> [class_id, x_center, y_center, width, height]
    annotations = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center = float(parts[1]) * w
                y_center = float(parts[2]) * h
                width = float(parts[3]) * w
                height = float(parts[4]) * h

                # Convert to xmin, ymin, xmax, ymax
                xmin = x_center - width / 2
                ymin = y_center - height / 2
                xmax = x_center + width / 2
                ymax = y_center + height / 2

                annotations.append({
                    'class_id': class_id,
                    'xmin': xmin,
                    'ymin': ymin,
                    'xmax': xmax,
                    'ymax': ymax
                })

    # Calculate the number of slices
    stride = int(slice_size * (1 - overlap_ratio))
    num_x_slices = max(1, int(np.ceil((w - slice_size) / stride)) + 1)
    num_y_slices = max(1, int(np.ceil((h - slice_size) / stride)) + 1)

    slice_idx = 0

    # Create slices
    for y_idx in range(num_y_slices):
        for x_idx in range(num_x_slices):
            # Calculate coordinates for this slice
            x_start = min(x_idx * stride, w - slice_size)
            y_start = min(y_idx * stride, h - slice_size)
            x_end = x_start + slice_size
            y_end = y_start + slice_size

            # Handle edge cases
            x_start = max(0, x_end - slice_size)
            y_start = max(0, y_end - slice_size)
            x_end = min(x_start + slice_size, w)
            y_end = min(y_start + slice_size, h)

            # Extract slice
            slice_img = image[y_start:y_end, x_start:x_end]
            slice_h, slice_w = slice_img.shape[:2]

            # Create labels for the slice
            slice_annotations = []
            for anno in annotations:
                # Check if annotation intersects with the slice
                if (anno['xmin'] < x_end and anno['xmax'] > x_start and
                    anno['ymin'] < y_end and anno['ymax'] > y_start):

                    # Clip the bounding box to the slice
                    xmin_clipped = max(0, anno['xmin'] - x_start)
                    ymin_clipped = max(0, anno['ymin'] - y_start)
                    xmax_clipped = min(slice_w, anno['xmax'] - x_start)
                    ymax_clipped = min(slice_h, anno['ymax'] - y_start)

                    width_clipped = xmax_clipped - xmin_clipped
                    height_clipped = ymax_clipped - ymin_clipped

                    # Check if the area is significant
                    original_area = (anno['xmax'] - anno['xmin']) * (anno['ymax'] - anno['ymin'])
                    clipped_area = width_clipped * height_clipped

                    # Only include if the intersection is significant (at least 30% of original bbox is visible)
                    if clipped_area >= 0.3 * original_area:
                        # Convert back to YOLO format (class_id, x_center, y_center, width, height)
                        x_center = (xmin_clipped + xmax_clipped) / (2 * slice_w)
                        y_center = (ymin_clipped + ymax_clipped) / (2 * slice_h)
                        width = (xmax_clipped - xmin_clipped) / slice_w
                        height = (ymax_clipped - ymin_clipped) / slice_h

                        slice_annotations.append(f"{anno['class_id']} {x_center} {y_center} {width} {height}")

            if slice_annotations:
                slice_file = f"{base_name}_slice_{slice_idx}.jpg"
                cv2.imwrite(os.path.join(output_images_dir, slice_file), slice_img)

                slice_label_file = f"{base_name}_slice_{slice_idx}.txt"
                with open(os.path.join(output_labels_dir, slice_label_file), 'w') as f:
                    f.write('\n'.join(slice_annotations))

                slice_idx += 1

def apply_albumentations_augmentations(data_yaml_path):
    """
    Apply albumentations augmentations to the dataset.

    Args:
        data_yaml_path: Path to the data.yaml file
    """
    data_config = load_yaml(data_yaml_path)
    train_dir = data_config['train']

    # If train_dir is a relative path, make it absolute based on the yaml location
    if not os.path.isabs(train_dir):
        base_dir = os.path.dirname(data_yaml_path)
        train_dir = os.path.join(base_dir, train_dir)

    images_dir = train_dir
    if os.path.isdir(os.path.join(train_dir, "images")):
        images_dir = os.path.join(train_dir, "images")

    labels_dir = images_dir.replace("images", "labels")

    # Augmentation pipeline
    aug_transforms = A.Compose([
        A.RandomRotate90(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.HueSaturationValue(p=0.2),
        A.RandomGamma(gamma_limit=(60, 115), p=0.3),
        A.GaussNoise(p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    images = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for img_file in tqdm(images, desc="Applying augmentations"):
        base_name = os.path.splitext(img_file)[0]
        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, f"{base_name}.txt")

        if not os.path.exists(label_path):
            continue

        image = cv2.imread(img_path)
        h, w = image.shape[:2]

        # Read labels
        bboxes = []
        class_labels = []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:5])
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)

        if not bboxes:
            continue

        # Apply augmentations 2 times
        for i in range(2):
            try:
                augmented = aug_transforms(image=image, bboxes=bboxes, class_labels=class_labels)
                aug_image = augmented['image']
                aug_bboxes = augmented['bboxes']
                aug_class_labels = augmented['class_labels']

                # Save augmented image
                aug_img_file = f"{base_name}_aug_{i}.jpg"
                cv2.imwrite(os.path.join(images_dir, aug_img_file), aug_image)

                # Save augmented labels
                aug_label_file = f"{base_name}_aug_{i}.txt"
                with open(os.path.join(labels_dir, aug_label_file), 'w') as f:
                    for j, bbox in enumerate(aug_bboxes):
                        f.write(f"{int(aug_class_labels[j])} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
            except Exception as e:
                print(f"Error augmenting {img_file}: {e}")
                continue


def evaluate_with_sahi(model_path, data_yaml_path, slice_size=640, overlap_ratio=0.2):
    """
    Evaluate the model with SAHI.

    Args:
        model_path: Path to the trained model weights
        data_yaml_path: Path to the data.yaml file
        slice_size: Size of the slices
        overlap_ratio: Overlap ratio between slices
    """
    data_config = load_yaml(data_yaml_path)
    val_dir = data_config['val']

    # If val_dir is a relative path, make it absolute based on the yaml location
    if not os.path.isabs(val_dir):
        base_dir = os.path.dirname(data_yaml_path)
        val_dir = os.path.join(base_dir, val_dir)

    # Load the model using SAHI
    detection_model = AutoDetectionModel.from_pretrained(
        model_type='ultralytics',
        model_path=model_path,
        confidence_threshold=0.3,
        device=device
    )

    # Get validation images
    if os.path.isdir(os.path.join(val_dir, "images")):
        val_dir = os.path.join(val_dir, "images")

    images = [os.path.join(val_dir, f) for f in os.listdir(val_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Evaluate with SAHI
    for img_path in tqdm(images, desc="Evaluating with SAHI"):
        image = read_image(img_path)
        img_name = os.path.basename(img_path).split('.')[0]

        standard_prediction = get_prediction(
            image=image,
            detection_model=detection_model,
        )
        standard_prediction.export_visuals(export_dir="out/standard/", file_name=img_name, rect_th=2)

        sliced_prediction = get_sliced_prediction(
            image=image,
            detection_model=detection_model,
            slice_height=slice_size,
            slice_width=slice_size,
            overlap_height_ratio=overlap_ratio,
            overlap_width_ratio=overlap_ratio
        )

        print(f"\nImage: {img_name}")
        print(f"Standard prediction count: {len(standard_prediction.object_prediction_list)}")
        print(f"SAHI sliced prediction count: {len(sliced_prediction.object_prediction_list)}")
        sliced_prediction.export_visuals(export_dir="out/sahi/", file_name=img_name, rect_th=2)


def main():
    import argparse

    parser = argparse.ArgumentParser(description="SAHI implementation for YOLO with small dataset")
    parser.add_argument('--data', type=str, default='datasets/erick/data.yaml',
                        help='Path to data.yaml file')
    parser.add_argument('--output', type=str, default='datasets/slots',
                        help='Output directory for the sliced SAHI dataset')
    parser.add_argument('--slice-size', type=int, default=640,
                        help='Size of the slices')
    parser.add_argument('--overlap-ratio', type=float, default=0.2,
                        help='Overlap ratio between slices')
    parser.add_argument('--skip-slicing', action='store_true',
                        help='Skip SAHI slicing')
    parser.add_argument('--skip-augmentations', action='store_true',
                        help='Skip Albumentations augmentations')

    args = parser.parse_args()

    # Create sliced (SAHI) dataset
    if not args.skip_slicing:
        data_yaml_path = create_sahi_dataset(
            args.data,
            args.output,
            args.slice_size,
            args.overlap_ratio
        )
    else:
        data_yaml_path = args.data

    if not args.skip_augmentations:
        print("Applying additional augmentations...")
        apply_albumentations_augmentations(data_yaml_path)

    print("Done!")

if __name__ == "__main__":
    main()
    # evaluate_with_sahi(r"C://Users/rocha/Documents/NACRE/sahi_improved_yolo/yolov8n_mixed_resolution/weights/best.pt", r"C://Users/rocha/Documents/NACRE/datasets/erick/data.yaml", 640, 0.2)

In [ ]:
import utils

output_dir = "/content/datasets/erick_sahi_augmented"
new_data_yaml = utils.create_sahi_dataset(data_yaml_path, output_dir)
# utils.apply_albumentations_augmentations("C://Users/rocha/Documents/NACRE/datasets/erick_sahi_augmented/data.yaml")

model = YOLO("yolov8n.pt")

config = {
    "data": f"/content/{new_data_yaml}",
    "epochs": 20,
    "imgsz": 640,
    "batch": 8,
    "workers": 0,
    "device": device,
    "freeze": list(range(11)), # 0 ... 10
    "augment": True,
    "mosaic": 1.0,
    "mixup": 0.5,
    "scale": 0.5,
    "fliplr": 0.5,
    "flipud": 0.2,
    "hsv_h": 0.015,
    "hsv_s": 0.9,
    "hsv_v": 0.9,
    "patience": 30,
    "seed": 42,
    "single_cls": True,
    "project": "slots",
    "name": "sahi",
}

## Training

Run the block bellow to train the new version

#5.&nbsp;Results Discution


In [ ]:
results = model.train(**config)

As shown in the figure below, which presents the model metrics, the training losses (`box_loss`, `cls_loss`, and `df_loss`) consistently decrease over the epochs, indicating that the model is effectively learning to locate and classify objects. The validation losses (`val/box_loss`, `val/cls_loss`, `val/df_loss`) also decline, albeit at a slightly slower pace, suggesting good generalization.

Meanwhile, the performance metrics (`Precision`, `Recall`, `mAP@50`, `mAP@50-95`) progressively increase and stabilize at high values. This indicates that the model is improving in terms of precision (correct predictions). However, the `mAP@50-95` still shows a drop, which suggest difficulties in precisely localizing objects at higher IoU thresholds.

In [ ]:
results_path = Path(f"/content/{config['project']}/{config['name']}/")
results = cv2.imread(str(results_path / "results.png"))
plt.figure(figsize=(14, 10))
plt.imshow(cv2.cvtColor(results, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# slicing over the original val (5 images - full size)
utils.evaluate_with_sahi(f"/content/{config['project']}/{config['name']}/weights/best.pt", data_yaml_path)

As can be seen in the figure bellow (after running the block), applying SAHI significantly improved class identification accuracy, increasing from approximately 50% to 80%. Additionally, detections became more consistent, reducing false positives and further demonstrating the effectiveness of the technique. However, it was observed that images with better lighting conditions yielded more stable results, suggesting that illumination plays a crucial role in detection performance and can be further improved.

## Further Improvement Sugestions
1. **Data Improvements**:
   - Increase dataset size
   - Add more challenging scenarios (occlusions, varying lighting)
   - Improve annotation quality to reduce label incosistence
2. **Model Improvements**:
   - Larger YOLO variants (yolov8s/yolov8x/yolo11s/yolo11l)
   - Explore R-CNN models for higher accuracy (like detectron2)
3. **Training Improvements**:
   - Hyperparameter optimization (learning rate, weight decay)
   - Tunning with a wide space range (Computational Heavy)
   - Longer training with progressive resizing
4. **Post-processing**:
   - Optimize confidence thresholds to reduce false positives
   - Implement/Use test-time augmentation (TTA)

In [ ]:
# Load the SAHI results
sahi_image_paths = glob.glob(os.path.join("out/sahi/", "*.*"))
sahi_image_paths = [p for p in sahi_image_paths if p.lower().endswith(('.png', '.jpg', '.jpeg'))]
sahi_img = cv2.imread(sahi_image_paths[0])

# Load the standard YOLO results
stantard_image_paths = glob.glob(os.path.join("out/standard/", "*.*"))
stantard_image_paths = [p for p in stantard_image_paths if p.lower().endswith(('.png', '.jpg', '.jpeg'))]
stantard_img = cv2.imread(stantard_image_paths[0])

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(cv2.cvtColor(sahi_img, cv2.COLOR_BGR2RGB))
axes[0].set_title(f"SAHI: {os.path.basename(sahi_image_paths[0])}", fontsize=12)
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(stantard_img, cv2.COLOR_BGR2RGB))
axes[1].set_title(f"Standard YOLO: {os.path.basename(stantard_image_paths[0])}", fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.suptitle(f"SAHI vs Standard YOLO Inference Comparison", y=0.98)
plt.show()